In [19]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
from textblob import TextBlob
from sklearn.linear_model import LogisticRegression
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
nlp = spacy.load('en')

In [3]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [4]:
def build_df(text1, text2):
    # Parse the cleaned novels. This can take a bit.
    text1 = text_cleaner(text1)
    text2 = text_cleaner(text2)
    
    text1_doc = nlp(text1)
    text2_doc = nlp(text2)
    
    # Group into sentences.
    text1_sents = [[sent, "Author1"] for sent in text1_doc.sents]
    text2_sents = [[sent, "Author2"] for sent in text2_doc.sents]

    # Combine the sentences from the two novels into one data frame.
    sentences = pd.DataFrame(text1_sents + text2_sents)
    
    # Set up the bags.
    text1words = bag_of_words(text1_doc)
    text2words = bag_of_words(text2_doc)

    # Combine bags to create a set of unique words.
    common_words = set(text1words + text2words)
    
    # Create our data frame with features. This can take a while to run.
    word_counts = bow_features(sentences, common_words)

    return word_counts

In [5]:
def count_pos(sent, pos):
    thissent = ' '.join(sent)
    sentenc = nlp(thissent)
    y = 0
    for token in sentenc:
        if token.pos_ == pos:
            y+=1
    return y

In [6]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        #Count the Parts of Speech
        partsofspeech = ['ADJ', 'ADP', 'ADV','CCONJ','DET','NOUN','PART','PRON', 'VERB', 'PUNC']
        for part in partsofspeech:
            df.loc[i, part]=count_pos(words, part)
        
        #Calculate the Polarity
        s = TextBlob(' '.join(words))
        df.loc[i, 'polarity'] = s.sentiment.polarity
        df.loc[i, 'subjectivity'] = s.sentiment.subjectivity
        
        #Calculate the average word length
        if(len(words)!= 0):
            df.loc[i, 'avg_wrd_len'] = np.mean([len(words[c]) for c in range (len(words))])
        else:
            df.loc[i, 'avg_wrd_len'] = 0
        
        #Calculate the words per sentence
        df.loc[i,'wrdspersent'] = len(words)

        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
        
    return df

In [7]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [8]:
parents = gutenberg.raw('edgeworth-parents.txt')
busterbrown = gutenberg.raw('burgess-busterbrown.txt')

In [9]:
featuresset = build_df(parents, busterbrown)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500
Processing row 6000
Processing row 6500
Processing row 7000
Processing row 7500
Processing row 8000
Processing row 8500
Processing row 9000


In [10]:
featuresset.shape

(9315, 2480)

In [11]:
featuresset.head()

,table,montague,takes,talb,retire,discouraged,scrape,sweepstakes,quantity,fin,...,DET,NOUN,PART,PRON,VERB,PUNC,polarity,subjectivity,avg_wrd_len,wrdspersent
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,4.0,0.0,0.0,0.0,0.0,-0.004545,0.433333,5.333333,9.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,7.0,0.0,0.0,3.0,0.0,0.162500,0.656250,5.066667,15.0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.100000,0.200000,3.750000,4.0
4,0,0,0,0,0,0,0,0,0,0,...,0.0,15.0,1.0,0.0,8.0,0.0,-0.171875,0.475000,4.714286,28.0


In [12]:
Y = featuresset['text_source']
X = np.array(featuresset.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4)

In [13]:
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(5589, 2478) (5589,)
Training set score: 0.972803721596

Test set score: 0.958132045089


In [14]:
score = cross_val_score(cv=10, estimator=lr, X=X, y=Y)

In [15]:
score.mean()

0.95662619915822655

# BOW Results
Using my version of bag of words I achieved a 95.6% accuracy on predicting the authors of the work.  Lets see how we do with idf.

In [16]:
parents=gutenberg.raw('edgeworth-parents.txt')
parents_doc = nlp(parents)
parents_sents = [sent.string for sent in parents_doc.sents]

busterbrown=gutenberg.raw('burgess-busterbrown.txt')
busterbrown_doc = nlp(busterbrown)
busterbrown_sents = [sent.string for sent in busterbrown_doc.sents]

In [17]:
parentsdf = pd.DataFrame(parents_sents, columns = ['sentences'])
busterbrowndf = pd.DataFrame(busterbrown_sents, columns = ['sentences'])
parentsdf['Author'] = 1
busterbrowndf['Author'] = 0
df = pd.concat([parentsdf, busterbrowndf])

In [20]:
tfidfv = TfidfVectorizer(min_df=1,stop_words='english')

In [21]:
x_traincv=tfidfv.fit_transform(df['sentences'])
score = cross_val_score(X=x_traincv, y = df['Author'], cv = 10, estimator=lr)

In [22]:
score.mean()

0.94217377481417652

In [23]:
score

array([ 0.94989107,  0.9476554 ,  0.94983642,  0.94329335,  0.9476554 ,
        0.94868996,  0.9268559 ,  0.93340611,  0.94541485,  0.9290393 ])